# Introduction

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

In [93]:
data = pd.read_csv('AmesHousing.tsv',delimiter='\t')

In [94]:
def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area','SalePrice']] # return dataframe here

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    
    # Include the numeric types (int and float here)
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    # generate features and linear regression model
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'],predictions)
    rmse = np.sqrt(mse)
    return rmse

transformed_data = transform_features(data)
filtered_data = select_features(transformed_data)
rmse_data = train_and_test(filtered_data)
rmse_data

57088.25161263909

# Feature Engineering

### All columns
* Drop columns that contain 5% or more missing values
* Numerical columns fill in the most popular value for that column
* Text column drop 1 missing value

In [95]:
# Get number of missing values
missing_values = data.isnull().sum()
print(missing_values[missing_values != 0]) # What columns having missing values

Lot Frontage       490
Alley             2732
Mas Vnr Type        23
Mas Vnr Area        23
Bsmt Qual           80
Bsmt Cond           80
Bsmt Exposure       83
BsmtFin Type 1      80
BsmtFin SF 1         1
BsmtFin Type 2      81
BsmtFin SF 2         1
Bsmt Unf SF          1
Total Bsmt SF        1
Electrical           1
Bsmt Full Bath       2
Bsmt Half Bath       2
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      159
Garage Finish      159
Garage Cars          1
Garage Area          1
Garage Qual        159
Garage Cond        159
Pool QC           2917
Fence             2358
Misc Feature      2824
dtype: int64


In [96]:
# Filter to columns that have more than 5% missing values
drop_missing_columns = missing_values[(missing_values > len(data)/20)].sort_values()
drop_missing_columns

Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Lot Frontage      490
Fireplace Qu     1422
Fence            2358
Alley            2732
Misc Feature     2824
Pool QC          2917
dtype: int64

In [97]:
# drop these columns from data frame
data = data.drop(drop_missing_columns.index, axis = 1)

# Check what has missing values
data.isnull().sum()

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Area            0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Mas Vnr Type       23
Mas Vnr Area       23
Exter Qual          0
Exter Cond          0
Foundation          0
Bsmt Qual          80
                   ..
Electrical          1
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Bsmt Full Bath      2
Bsmt Half Bath      2
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Functional          0
Fireplaces

In [98]:
# Get the text columns that have missing values (in descending order)
text_missing = data.select_dtypes(include='object').isnull().sum().sort_values(ascending=False)
text_missing

Bsmt Exposure     83
BsmtFin Type 2    81
BsmtFin Type 1    80
Bsmt Qual         80
Bsmt Cond         80
Mas Vnr Type      23
Electrical         1
Utilities          0
Lot Config         0
Land Slope         0
Roof Matl          0
Land Contour       0
Lot Shape          0
Street             0
Neighborhood       0
Condition 1        0
Condition 2        0
Bldg Type          0
House Style        0
Roof Style         0
Sale Condition     0
Exterior 1st       0
Exterior 2nd       0
Sale Type          0
Exter Qual         0
Exter Cond         0
Foundation         0
Heating            0
Heating QC         0
Central Air        0
Kitchen Qual       0
Functional         0
Paved Drive        0
MS Zoning          0
dtype: int64

In [99]:
# Drop these columns
text_missing_drop = text_missing[(text_missing > 0)]
data = data.drop(text_missing_drop.index, axis = 1)
data.isnull().sum()

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Area            0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Mas Vnr Area       23
Exter Qual          0
Exter Cond          0
Foundation          0
BsmtFin SF 1        1
BsmtFin SF 2        1
                   ..
Central Air         0
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Bsmt Full Bath      2
Bsmt Half Bath      2
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Functional          0
Fireplaces

In [100]:
# Compute numerical missing columns
num_missing = data.select_dtypes(include=['integer','float']).isnull().sum()
fix_num_cols = num_missing[(num_missing < len(data)/20) & (num_missing > 0)].sort_values()
fix_num_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [101]:
# Compute the most common value (mode) in fix_num_cols
replacement_values_dict = data[fix_num_cols.index].mode().to_dict(orient='records')[0]
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [102]:
# Fill in numerical columns with replacement values
data = data.fillna(replacement_values_dict)

In [103]:
data.isnull().sum().value_counts() # Make sure there are no missing values

0    64
dtype: int64

In [104]:
# New features are years sold and years since remodeld
years_sold = data['Yr Sold'] - data['Year Built']
years_sold[years_sold < 0] # because this doesn't make any sense

2180   -1
dtype: int64

In [105]:
years_since_remod = data['Yr Sold'] - data['Year Remod/Add']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [106]:
# Create New Columns
data['Years Before Sale'] = years_sold
data['Years Since Remod'] = years_since_remod

# Drop rows with negative values
data = data.drop([1702,2180,2181],axis = 0)

# Don't need the original columns
data = data.drop(['Year Built','Year Remod/Add'], axis = 1)

In [107]:
# Drop columns that aren't required for machine learning from documentation
data = data.drop(['PID','Order'], axis = 1)

# Drop columns that leak final sale data
data = data.drop(['Mo Sold','Yr Sold','Sale Type','Sale Condition'], axis = 1)

In [108]:
def transform_features(df):
    missing_values = df.isnull().sum()
    drop_missing_columns = missing_values[(missing_values > len(data)/20)].sort_values()
    df = df.drop(drop_missing_columns.index, axis = 1)
    
    text_missing = df.select_dtypes(include='object').isnull().sum().sort_values(ascending=False)
    text_missing_drop = text_missing[(text_missing > 0)]
    df = df.drop(text_missing_drop.index, axis = 1)
    
    num_missing = df.select_dtypes(include=['integer','float']).isnull().sum()
    fix_num_cols = num_missing[(num_missing < len(data)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = data[fix_num_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - data['Year Built']
    years_since_remod = df['Yr Sold'] - data['Year Remod/Add']
    
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702,2180,2181],axis = 0)
    df = df.drop(['PID','Order','Mo Sold','Yr Sold','Sale Type','Sale Condition','Year Built','Year Remod/Add'], axis = 1)
    
    return df

def select_features(df):
    return df[['Gr Liv Area','SalePrice']] # return dataframe here

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    
    # Include the numeric types (int and float here)
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    # generate features and linear regression model
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'],predictions)
    rmse = np.sqrt(mse)
    return rmse

data = pd.read_csv('AmesHousing.tsv',delimiter='\t')
transformed_data = transform_features(data)
filtered_data = select_features(transformed_data)
rmse_data = train_and_test(filtered_data)
rmse_data

55275.36731241307

In [109]:
reduce_rmse = (55275.36731241307 - 57088.25161263909) / (57088.25161263909)
reduce_rmse_percent = reduce_rmse * 100
print('Percent Reduced', reduce_rmse_percent)

Percent Reduced -3.1755821014225165


Using feature engineering we have reduced the rmse error from 57088.25161263909 to 55275.36731241307 which is a reduction of 3.18 %

# Feature Selection

In [110]:
# Generate numerical dataframe
numerical_dataframe = transformed_data.select_dtypes(include=['int','float'])
numerical_dataframe.head(5)

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years Since Remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12


In [111]:
abs_corr_coefficients = numerical_dataframe.corr()['SalePrice'].abs().sort_values()
abs_corr_coefficients

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: Sale

This shows the correlation between SalePrice and other columns.  The closer to one the better however, for now let's do 0.4 as the threshold (as that will include correlations in the 0.5 to 0.8 range as well)

In [112]:
abs_corr_coefficients[abs_corr_coefficients > 0.4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [113]:
# filter the dataframe by removing columns with correlation coefficients less than the thresold of 0.4
transformed_data = transformed_data.drop(abs_corr_coefficients[abs_corr_coefficients < 0.4].index, axis = 1)

In [114]:
transformed_data.head()

,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,...,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Garage Cars,Garage Area,Paved Drive,SalePrice,Years Before Sale,Years Since Remod
0,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,TA,7,Typ,2,2.0,528.0,P,215000,50,50
1,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,...,TA,5,Typ,0,1.0,730.0,Y,105000,49,49
2,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,Gd,6,Typ,0,1.0,312.0,Y,172000,52,52
3,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,Ex,8,Typ,2,2.0,522.0,Y,244000,42,42
4,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,TA,6,Typ,1,2.0,482.0,Y,189900,13,12


In [115]:
transformed_data.columns

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Area',
       'Exter Qual', 'Exter Cond', 'Foundation', 'BsmtFin SF 1',
       'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air', '1st Flr SF',
       'Gr Liv Area', 'Full Bath', 'Kitchen Qual', 'TotRms AbvGrd',
       'Functional', 'Fireplaces', 'Garage Cars', 'Garage Area', 'Paved Drive',
       'SalePrice', 'Years Before Sale', 'Years Since Remod'],
      dtype='object')

The documentation found [here](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt) lists what are nominal, ordinal, discreet etc.

In [116]:
nominal_features = ['PID','MS SubClass','MS Zoning','Street','Alley','Land Contour','Lot Config','Neighborhood',
                    'Condition 1', 'Condition 2', 'Bldg Type'
                   ,'House Style','Roof Style','Roof Matl','Exterior 1st','Exterior 2nd','Mas Vnr Type'
                   ,'Foundation','Heating','Central Air','Garage Type','Misc Feature','Sale Type','Sale Condition']

questions to ask for keeping categories
* If a categorical column has hundreds of unique values (or categories), should you keep it? When you dummy code this column, hundreds of columns will need to be added back to the data frame.
* Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?

In [117]:
# Testing which columns to keep
transform_data_cat_col = []
for col in nominal_features:
    if col in transformed_data.columns:
        transform_data_cat_col.append(col)

# How many unique values for each column
unique_count = transformed_data[tranform_data_cat_col].apply(lambda column: len(column.value_counts())).sort_values()

In [118]:
unique_count # Get the count for each column in the dataframe

Street           2
Central Air      2
Land Contour     4
Lot Config       5
Bldg Type        5
Roof Style       6
Foundation       6
Heating          6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
Exterior 2nd    17
Neighborhood    28
dtype: int64

In [119]:
# Using a cutoff of 10 will drop columns that are not unique
non_unique_count = unique_count[unique_count > 10]
transformed_data = transformed_data.drop(non_unique_count.index, axis = 1)

In [120]:
transformed_data.columns

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Roof Style', 'Roof Matl',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation',
       'BsmtFin SF 1', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       '1st Flr SF', 'Gr Liv Area', 'Full Bath', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Garage Cars',
       'Garage Area', 'Paved Drive', 'SalePrice', 'Years Before Sale',
       'Years Since Remod'],
      dtype='object')

In [121]:
# Grab the rest of the text columns in the tranformed dataframe and convert to category
text_columns = transformed_data.select_dtypes(include=['object'])
for column in text_columns:
    transformed_data[column] = transformed_data[column].astype('category')

# Create dummny columns and add to the dataframe
transformed_data = pd.concat([transformed_data,pd.get_dummies(transformed_data.select_dtypes(include=['category']))],axis = 1).drop(text_columns, axis = 1)


In [122]:
def transform_features(df):
    missing_values = df.isnull().sum()
    drop_missing_columns = missing_values[(missing_values > len(data)/20)].sort_values()
    df = df.drop(drop_missing_columns.index, axis = 1)
    
    text_missing = df.select_dtypes(include='object').isnull().sum().sort_values(ascending=False)
    text_missing_drop = text_missing[(text_missing > 0)]
    df = df.drop(text_missing_drop.index, axis = 1)
    
    num_missing = df.select_dtypes(include=['integer','float']).isnull().sum()
    fix_num_cols = num_missing[(num_missing < len(data)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = data[fix_num_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - data['Year Built']
    years_since_remod = df['Yr Sold'] - data['Year Remod/Add']
    
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702,2180,2181],axis = 0)
    df = df.drop(['PID','Order','Mo Sold','Yr Sold','Sale Type','Sale Condition','Year Built','Year Remod/Add'], axis = 1)
    
    return df

def select_features(df, corr_threshold = 0.4, unique_threshold = 10):
    numerical_df = df.select_dtypes(include=['int','float'])
    abs_corr_coefficients = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coefficients[abs_corr_coefficients < corr_threshold].index, axis = 1)
    
    nominal_features = ['PID','MS SubClass','MS Zoning','Street','Alley','Land Contour','Lot Config','Neighborhood',
                    'Condition 1', 'Condition 2', 'Bldg Type'
                   ,'House Style','Roof Style','Roof Matl','Exterior 1st','Exterior 2nd','Mas Vnr Type'
                   ,'Foundation','Heating','Central Air','Garage Type','Misc Feature','Sale Type','Sale Condition']
    transform_data_cat_col = []
    
    for col in nominal_features:
        if col in df.columns:
            transform_data_cat_col.append(col)
    
    unique_count = df[tranform_data_cat_col].apply(lambda column: len(column.value_counts())).sort_values()
    non_unique_count = unique_count[unique_count > unique_threshold]
    df = df.drop(non_unique_count.index, axis = 1)
    
    text_columns = df.select_dtypes(include=['object'])
    for column in text_columns:
        df[column] = df[column].astype('category')
    df = pd.concat([transformed_data,pd.get_dummies(df.select_dtypes(include=['category']))],axis = 1).drop(text_columns, axis = 1)


    return df

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    
    # Include the numeric types (int and float here)
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    # generate features and linear regression model
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'],predictions)
    rmse = np.sqrt(mse)
    return rmse

data = pd.read_csv('AmesHousing.tsv',delimiter='\t')
transformed_data = transform_features(data)
filtered_data = select_features(transformed_data)
rmse_data = train_and_test(filtered_data)
rmse_data

56839.7263192318

Updating the select features function increased the rmse.  This means that we will need to look at the train_and_test function for improvements

# Train and Test

This will update our training and testing function using different k values
* k = 0 (do what we always were doing)
* k = 1 Cross validate with two rmses
* k > 0 Kfold

In [126]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer','float'])
    features = numeric_df.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    if k == 0:
        train = df[0:1460]
        test = df[1460:]
        lr.fit(train[features],train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'],predictions)
        rmse = np.sqrt(mse)
        return rmse
    
    if k == 1:
        shuffled_df = df.sample(frac=1)
        train = df[0:1460]
        test = df[1460:]
        
        lr.fit(train[features],train['SalePrice'])
        predictions_one = lr.predict(test[features])
        mse_one = mean_squared_error(test['SalePrice'],predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features],test['SalePrice'])
        predictions_two = lr.predict(train[features])
        mse_two = mean_squared_error(train['SalePrice'],predictions_two)
        rmse_two = np.sqrt(mse_two)
        avg_rmse = np.mean(rmse_one,rmse_two)
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        # using the kfold.split function to generate indices for training and testing
        for train_index, test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features],train['SalePrice'])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test['SalePrice'],predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

In [127]:
def transform_features(df):
    missing_values = df.isnull().sum()
    drop_missing_columns = missing_values[(missing_values > len(data)/20)].sort_values()
    df = df.drop(drop_missing_columns.index, axis = 1)
    
    text_missing = df.select_dtypes(include='object').isnull().sum().sort_values(ascending=False)
    text_missing_drop = text_missing[(text_missing > 0)]
    df = df.drop(text_missing_drop.index, axis = 1)
    
    num_missing = df.select_dtypes(include=['integer','float']).isnull().sum()
    fix_num_cols = num_missing[(num_missing < len(data)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = data[fix_num_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - data['Year Built']
    years_since_remod = df['Yr Sold'] - data['Year Remod/Add']
    
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702,2180,2181],axis = 0)
    df = df.drop(['PID','Order','Mo Sold','Yr Sold','Sale Type','Sale Condition','Year Built','Year Remod/Add'], axis = 1)
    
    return df

def select_features(df, corr_threshold = 0.4, unique_threshold = 10):
    numerical_df = df.select_dtypes(include=['int','float'])
    abs_corr_coefficients = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coefficients[abs_corr_coefficients < corr_threshold].index, axis = 1)
    
    nominal_features = ['PID','MS SubClass','MS Zoning','Street','Alley','Land Contour','Lot Config','Neighborhood',
                    'Condition 1', 'Condition 2', 'Bldg Type'
                   ,'House Style','Roof Style','Roof Matl','Exterior 1st','Exterior 2nd','Mas Vnr Type'
                   ,'Foundation','Heating','Central Air','Garage Type','Misc Feature','Sale Type','Sale Condition']
    transform_data_cat_col = []
    
    for col in nominal_features:
        if col in df.columns:
            transform_data_cat_col.append(col)
    
    unique_count = df[tranform_data_cat_col].apply(lambda column: len(column.value_counts())).sort_values()
    non_unique_count = unique_count[unique_count > unique_threshold]
    df = df.drop(non_unique_count.index, axis = 1)
    
    text_columns = df.select_dtypes(include=['object'])
    for column in text_columns:
        df[column] = df[column].astype('category')
    df = pd.concat([transformed_data,pd.get_dummies(df.select_dtypes(include=['category']))],axis = 1).drop(text_columns, axis = 1)


    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer','float'])
    features = numeric_df.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    if k == 0:
        train = df[0:1460]
        test = df[1460:]
        lr.fit(train[features],train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'],predictions)
        rmse = np.sqrt(mse)
        return rmse
    
    if k == 1:
        shuffled_df = df.sample(frac=1)
        train = df[0:1460]
        test = df[1460:]
        
        lr.fit(train[features],train['SalePrice'])
        predictions_one = lr.predict(test[features])
        mse_one = mean_squared_error(test['SalePrice'],predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features],test['SalePrice'])
        predictions_two = lr.predict(train[features])
        mse_two = mean_squared_error(train['SalePrice'],predictions_two)
        rmse_two = np.sqrt(mse_two)
        avg_rmse = np.mean(rmse_one,rmse_two)
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        # using the kfold.split function to generate indices for training and testing
        for train_index, test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features],train['SalePrice'])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test['SalePrice'],predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

data = pd.read_csv('AmesHousing.tsv',delimiter='\t')
transformed_data = transform_features(data)
filtered_data = select_features(transformed_data)
rmse_data = train_and_test(filtered_data, k=4)
rmse_data

[28934.08132133791, 24407.65443252987, 24835.647176233655, 37442.670350908]


28905.013320252358

Our average rmse has dropped significantly from where we started from 57088.25161263909 to 28905.013320252358 showing the benefit of using higher k values and KFold cross validation over holdout validation